<a href="https://colab.research.google.com/github/lolonao/code-playground/blob/main/Google%E3%81%AE%E6%A4%9C%E7%B4%A2%E7%B5%90%E6%9E%9C%E3%82%92%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%shell

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF


apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg


cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.qyRIW4Q0oe/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.tkWWyGQfv1/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.OOU9OnGvje/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [3]:
!apt-get update
!apt-get install chromium chromium-driver
!pip install selenium
!pip install requests==2.28.2 bs4==0.0.1 beautifulsoup4

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:8 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:9 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:12 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:14 http://deb.debian.org/d

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/")

Mounted at /content/drive


In [ ]:
"""
ボタンを自動でクリックして全ての商品情報を取得し、表示する
"""
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import io
from PIL import Image
from urllib.request import urlopen as uReq

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(10)

# ブラウザを起動し、Amazon.co.jpの検索結果ページを開く
# driver = webdriver.Chrome()
driver.get('https://www.amazon.co.jp/s?k=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E+made+in+Japan&i=kitchen&rh=n%3A3828871%2Cp_89%3A%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E%28PEARL+METAL%29%2Cp_n_price_fma%3A401051011%2Cp_76%3A2227292051&dc&language=en&ds=v1%3AR8ANb3RXxPQYvxkRqS%2Fb3M%2FG4rUW2XhNt2fZ%2BfuQwXA&crid=18CIBDBUS6B4L&qid=1679524003&rnid=3428066051&sprefix=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E+made+in+japan%2Ckitchen%2C159&ref=sr_nr_p_90_1')

pagenum = 1

# すべての商品情報を取得する
while True:
    # 商品情報が表示される要素を取得
    # products = driver.find_elements(By.XPATH, '//*[@data-component-type="s-search-result"]')
    xpath = '//*[@data-component-type="s-search-result"]'
    # products = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="search"]/div[1]/div[1]/div/span[1]/div')))
    products = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, xpath)))


    # 各商品の情報を取得
    for product in products:
         # 商品タイトルとURLを取得
        title_element = product.find_element(By.XPATH, './/h2/a')
        title = title_element.text
        url = title_element.get_attribute('href')
        print(title)
        print(url)
        
        # 価格を取得
        try:
            price_element = product.find_element(By.XPATH, './/span[@class="a-price-whole"]')
            price = price_element.text
            print(price)
        except:
            print("価格情報はありません")
        
        # 画像URLを取得
        try:
            img_element = product.find_element(By.XPATH, './/img')
            img_url = img_element.get_attribute('src')
            print(img_url)
        except:
            print("画像URLはありません")
        
        print('------------')
        time.sleep(3)

    # 「次へ」ボタンがあればクリックして次のページに進む
    try:
        time.sleep(10)
        # next_button = driver.find_element(By.XPATH, '//*[@id="search"]/div[1]/div[1]/div/span[1]/div[1]/div[last()]/div/div/span/a[3]')
        # next_button.click()
        next_button = driver.find_element(By.CSS_SELECTOR, "a.s-pagination-next")
        next_button.click()
        # wait.until(EC.staleness_of(items[0]))
        pagenum += 1
        print(f"\n{pagenum}ページへ...\n")
        time.sleep(3)
    except:
        print("最後のページです")
        break

In [6]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

# from selenium import webdriver
# from selenium.webdriver.common.by import By
import time
import requests
import io
from PIL import Image
from urllib.request import urlopen as uReq

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(10)

try:
    # Googleのトップページを開く
    driver.get("https://www.google.com/")
except Exception as e:
    print(f"エラー: {e}")

time.sleep(10)


# 検索ボックスを探す
search_box = driver.find_element(By.NAME, "q")

# 検索ボックスにキーワードを入力する
search_box.send_keys("used car dealership in tonga")

# Enterキーを押す
search_box.submit()

# 検索結果を表示するページに遷移するまで待機する
time.sleep(3)

# 検索結果のタイトルとURLを取得する
for page in range(1, 11):  # 1から3ページまで取得する（停止するページ数を3ページに設定）
    results = driver.find_elements(By.CSS_SELECTOR, "div.g")
    for result in results:
        try:
            title = result.find_element(By.CSS_SELECTOR, "h3").text
            url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            print(title)
            print(url)
            print('---------')
        except:
            pass
    # 次のページの検索結果を取得する
    try:
        next_button = driver.find_element(By.XPATH, "//a[@id='pnnext']")
        next_button.click()
        # 検索結果を表示するページに遷移するまで待機する
        time.sleep(30)
    except:
        break  # 次のページがない場合はループを抜ける

# ブラウザを閉じる
driver.quit()



Asco Motors Tonga | Vehicle Dealer
https://www.ascomotorstonga.com/
---------
Cars for sale in tonga
https://www.tongacars.com/buy-cars
---------
Alma Motors Tonga - Home
https://www.facebook.com/almamotors1/
---------
Japanese Used Cars For Sale In Tonga
https://www.autocraftjapan.com/car-country/Tonga
---------
Tonga Cheap Car Import , Nukualofa, Tonga | Used Cars ...
https://www.japanesecartrade.com/tonga/
---------
BE FORWARD Tonga
https://www.beforward.jp/beforward_tonga
---------
Asco Motors Tonga | New Toyota Nuku'alofa
https://www.toyota-tonga.com/
---------

https://www.autotrader.com/car-news/worlds-largest-car-dealer-southern-california-amazing-267234#:~:text=If%20you've%20ever%20wanted,truly%2C%20unbelievably%2C%20absurdly%20massive.
---------

https://finance.yahoo.com/news/does-carmax-vroom-carvana-pay-174500900.html#:~:text=Regardless%20of%20the%20age%20of,those%20from%202009%20and%20earlier.
---------


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
wait = WebDriverWait(driver, 10)

try:
    # Googleのトップページを開く
    # driver.get("https://www.google.com/")
    # Bingのトップページを開く
    driver.get("https://www.bing.com/")
except Exception as e:
    print(f"エラー: {e}")


# 検索ボックスを探す
search_box = driver.find_element(By.NAME, "q")

# 検索ボックスにキーワードを入力する

keywords = "used car dealer dealer in Papua New Guinea"
search_box.send_keys(keywords)

# Enterキーを押す
search_box.submit()

# 検索結果を表示するページに遷移するまで待機する
wait = WebDriverWait(driver, 20)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.b_algo")))

# 検索結果のタイトルとURLを取得する
for page in range(1, 11):
    print(f"ページ{page}")
    results = driver.find_elements(By.CSS_SELECTOR, "li.b_algo")
    for result in results:
        try:
            title = result.find_element(By.CSS_SELECTOR, "h2").text
            url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            print(title)
            print(url)
            # print('---------')
        except Exception as e:
            print(f"エラー: {e}")
            pass


    # 次のページの検索結果を取得する
    try:
        sel = ".sb_pagN"
        next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, sel)))
        while True:
            try:
                # Colab上では --headless オプションが有効だと機能しないっぽい
                next_button.location_once_scrolled_into_view
                # driver.execute_script("arguments[0].scrollIntoView();", next_button)
                next_button.click()
                break
            except ElementClickInterceptedException:
                continue
        # 検索結果を表示するページに遷移するまで待機する
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.b_algo")))
    except Exception as e:
        print(f"エラー: {e}")
        break  # 次のページがない場合はループを抜ける


ページ1
PNG Motors - New and Used Cars Vehicles in Papua …
https://pngmotors.com/
USED CAR DEALERS in Papua New Guinea - PNG …
https://www.pngbusinessdirectory.com/category/used-car-dealers
Used Vehicles For Sale | Ela Motors PNG
https://www.elamotorspng.com/used-vehicles-png/
KR Motors Ltd., Pt Moresby, Papua New Guinea | Used Car Dealer
https://www.japanesecartrade.com/KRMotors/
ページ2
PNG Motors - New and Used Cars Vehicles in Papua …
https://pngmotors.com/
Boroko Motors | Who Do You Call - Boroko Motors Tasol
https://borokomotors.com/
Used Vehicles For Sale | Ela Motors PNG
https://www.elamotorspng.com/used-vehicles-png/
KR Motors Ltd., Pt Moresby, Papua New Guinea | Used Car Dealer
https://www.japanesecartrade.com/KRMotors/
KR Motors Ltd., Pt Moresby, Papua New Guinea
https://www.japanesecartrade.com/KRMotors/about-us.html
Papua New Guinea | AUTOREC
https://www.autorec.co.jp/import/papua-new-guinea
MOTOR CAR DEALERS - NEW AND USED in PNG - PNG …
https://www.pngbusinessdirectory.com/cat